In [1]:
import heapq  # Importing heapq to implement the priority queue for A* algorithm
import random  # Importing random to place the house randomly
import math    # Importing math for distance calculations

# Function to create a grid based on user-defined size and randomly place the house
def create_grid(size):
    grid = [[' ' for _ in range(size)] for _ in range(size)]  # Create an empty grid filled with spaces
    grid[0][0] = 'S'  # 'S' marks the starting point at the top-left corner (0,0)

    # Place the house ('H') at a random position, ensuring it’s not placed at the start
    house_x, house_y = random.randint(0, size-1), random.randint(0, size-1)
    while (house_x, house_y) == (0, 0):  # Ensure the house is not at the start
        house_x, house_y = random.randint(0, size-1), random.randint(0, size-1)

    grid[house_x][house_y] = 'H'  # Place the house 'H'
    return grid, (house_x, house_y)  # Return the grid and house's position

# Function to add random obstacles to the grid
def add_obstacles(grid, num_obstacles):
    size = len(grid)
    for _ in range(num_obstacles):
        obstacle_x, obstacle_y = random.randint(0, size-1), random.randint(0, size-1)
        # Ensure obstacles are not placed on the start 'S' or house 'H'
        while grid[obstacle_x][obstacle_y] in ['S', 'H']:
            obstacle_x, obstacle_y = random.randint(0, size-1), random.randint(0, size-1)
        grid[obstacle_x][obstacle_y] = 'X'  # Place the obstacle 'X'
    return grid

# Function to check if a position is valid (within bounds and not blocked by an obstacle)
def is_valid_position(grid, x, y):
    size = len(grid)
    return 0 <= x < size and 0 <= y < size and grid[x][y] != 'X'  # Return true if position is valid

# Heuristic function: Calculates the straight-line (Euclidean) distance between the current node and the goal
def heuristic(a, b):
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

# A* algorithm function to find the shortest path
def a_star(grid, start, goal):
    size = len(grid)
    # Priority queue to keep track of nodes to explore (using heapq)
    open_list = []
    heapq.heappush(open_list, (0, start))  # Add the start node with priority 0

    # Dictionaries to store the cost to reach each node and the path taken
    g_score = {start: 0}  # Cost from start to current node
    parent = {start: None}  # To reconstruct the path

    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Possible directions: up, down, left, right

    while open_list:
        _, current = heapq.heappop(open_list)  # Get the node with the lowest f(n)

        # If we reach the goal (house), stop the search
        if current == goal:
            break

        # Explore neighboring positions (up, down, left, right)
        for direction in directions:
            next_x = current[0] + direction[0]  # Calculate next x-coordinate
            next_y = current[1] + direction[1]  # Calculate next y-coordinate
            next_state = (next_x, next_y)  # Form the next state (position)

            # Check if the next position is valid
            if is_valid_position(grid, next_x, next_y):
                tentative_g_score = g_score[current] + 1  # Cost of moving to the next position

                # If the next state has not been explored or we found a cheaper path to it
                if next_state not in g_score or tentative_g_score < g_score[next_state]:
                    g_score[next_state] = tentative_g_score  # Update the cost to reach this state
                    f_score = tentative_g_score + heuristic(next_state, goal)  # Calculate the total cost f(n)
                    heapq.heappush(open_list, (f_score, next_state))  # Add the state to the open list
                    parent[next_state] = current  # Set the current state as the parent of the next state

    # Reconstruct the path from start to goal
    path = []
    current = goal  # Start from the goal (house) and work backwards
    while current is not None:
        path.append(current)  # Add current position to the path
        current = parent[current]  # Move to the parent of the current state
    path.reverse()  # Reverse the path to start from the beginning (start to goal)

    return path

# Function to print the grid with lines and the path marked
def print_grid_with_path(grid, path):
    grid_with_path = [row.copy() for row in grid]  # Copy the original grid to preserve it
    for (x, y) in path:
        if grid_with_path[x][y] not in ['S', 'H']:  # Don't overwrite start 'S' and house 'H'
            grid_with_path[x][y] = '*'  # Mark the path with '*'

    # Print the grid with lines to separate cells clearly
    print("\nGrid with Path:")
    print('-' * (len(grid_with_path) * 4 + 1))  # Print top border line
    for row in grid_with_path:
        print('| ' + ' | '.join(row) + ' |')  # Print row with borders between cells
        print('-' * (len(grid_with_path) * 4 + 1))  # Print horizontal line after each row

# Main function to play the delivery game
def delivery_game():
    # Ask the user for grid size and number of obstacles
    size = int(input("Enter the grid size (e.g., 6 for a 6x6 grid): "))
    num_obstacles = int(input(f"Enter the number of obstacles (less than {size * size - 2}): "))  # Make sure obstacles are less than available spaces

    # Create the grid and place the house randomly
    grid, goal = create_grid(size)

    # Add random obstacles to the grid
    grid = add_obstacles(grid, num_obstacles)

    # Print the initial grid
    print("\nInitial Grid:")
    print_grid_with_path(grid, [])  # Empty path initially

    # Start the search using A* algorithm
    print("\nSearching for the house using A* algorithm...\n")
    path = a_star(grid, (0, 0), goal)  # Perform A* search

    # Print the final grid with the path marked
    print("\nPath to the House:")
    print_grid_with_path(grid, path)  # Show the path to the house

# Run the game
delivery_game()


Enter the grid size (e.g., 6 for a 6x6 grid): 5
Enter the number of obstacles (less than 23): 2

Initial Grid:

Grid with Path:
---------------------
| S | X |   |   |   |
---------------------
|   | X |   |   |   |
---------------------
|   |   |   |   |   |
---------------------
|   |   |   |   |   |
---------------------
|   |   | H |   |   |
---------------------

Searching for the house using A* algorithm...


Path to the House:

Grid with Path:
---------------------
| S | X |   |   |   |
---------------------
| * | X |   |   |   |
---------------------
| * | * |   |   |   |
---------------------
|   | * | * |   |   |
---------------------
|   |   | H |   |   |
---------------------
